In [3]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timezone, timedelta
import matplotlib.pyplot as plt
import plotly_express as px
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from app import bulk_insert_into_table, create_table

In [4]:
# Constants
SENSEBOX_ID = "6793f4e4c326f20007c34dd2"  # The ID of the SenseBox
API_URL_FORMAT_BOX = "https://api.opensensemap.org/boxes/{sensebox_id}?format={response_format}"
API_URL_FORMAT_SENSOR = "https://api.opensensemap.org/boxes/{sensebox_id}/data/{sensor_id}?from-date={fromDate}&to-date={toDate}&download=true&format=json"

# Function to create and return a dictionary {sensor name : sensor id}
def get_sensor_names_ids():
    # Construct the URL for the API request using the SenseBox ID and JSON format
    url = API_URL_FORMAT_BOX.format(sensebox_id=SENSEBOX_ID,response_format="json")

    # Ensure the API request is successful by checking the status code
    status_code = requests.get(url).status_code
    assert status_code, f"Failed fetching data from API. Status code: {status_code}"

    # Get the 'sensors' list from the JSON response
    sensors = requests.get(url).json().get("sensors")

    # Create a dictionary to hold sensor names and their corresponding IDs
    sensor_name_id = {}
    for sensor in sensors:
        # Format the sensor name: remove spaces and replace special characters
        name = sensor.get('title').replace(" ", "").replace(".", "_").replace("-", "_")
        _id = sensor.get('_id')  # Get the sensor ID
        sensor_name_id.update({name: _id})  # Add entry to the dictionary

    # Return the dictionary of sensor names and IDs
    return sensor_name_id

### So kann man die namen und ids als eine Liste haben

In [23]:
get_sensor_names_ids()

{'Temperatur': '6793f4e4c326f20007c34dd3',
 'rel_Luftfeuchte': '6793f4e4c326f20007c34dd4',
 'Beleuchtungsstärke': '6793f4e4c326f20007c34dd5',
 'UV_Intensität': '6793f4e4c326f20007c34dd6',
 'PM1': '6793f4e4c326f20007c34dd7',
 'PM2_5': '6793f4e4c326f20007c34dd8',
 'PM4': '6793f4e4c326f20007c34dd9',
 'PM10': '6793f4e4c326f20007c34dda'}

### Dict Ausgabe von der Funktion

In [24]:
get_sensor_names_ids()

{'Temperatur': '6793f4e4c326f20007c34dd3',
 'rel_Luftfeuchte': '6793f4e4c326f20007c34dd4',
 'Beleuchtungsstärke': '6793f4e4c326f20007c34dd5',
 'UV_Intensität': '6793f4e4c326f20007c34dd6',
 'PM1': '6793f4e4c326f20007c34dd7',
 'PM2_5': '6793f4e4c326f20007c34dd8',
 'PM4': '6793f4e4c326f20007c34dd9',
 'PM10': '6793f4e4c326f20007c34dda'}

### So kann man auf die einzelne Namen zugreifen

In [25]:
sensor_id = get_sensor_names_ids()["Temperatur"]
sensor_id

'6793f4e4c326f20007c34dd3'

In [5]:
from datetime import datetime, timezone, timedelta
def get_data(sensor_id:str,fromDate,toDate):

    #get URL with sensor_id and dates
    url = API_URL_FORMAT_SENSOR.format(sensebox_id=SENSEBOX_ID, sensor_id=sensor_id, fromDate=fromDate, toDate=toDate)
    
    #Make sure we get correct response
    status_code = requests.get(url).status_code
    assert status_code == 200, f"Failed fetching data from api {status_code}"
    
    #Get the json file of the sensor
    data = requests.get(url).json()
    data = [{'createdAt': item['createdAt'], 'value': item['value']} for item in data]
    return data


In [6]:
now = datetime.now(timezone.utc)
iso_now = now.isoformat().replace('+00:00','Z')

two_weeeks_ago = datetime.now(timezone.utc) - timedelta(weeks=3) 
iso_two_weeks_ago = two_weeeks_ago.isoformat().replace('+00:00','Z')

iso_now, iso_two_weeks_ago

data = get_data(sensor_id="6793f4e4c326f20007c34dd3",fromDate=iso_two_weeks_ago,toDate=iso_now)

### create_dataframe function mit inputs sensor_id und from- und toDate die einen default Wert haben

# DATABANK CONNECTION

In [16]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host="localhost",      
    port=5432,
    dbname="dash_db",
    user="dashuser",
    password="dashpassword"
)

# query = "SELECT * FROM public.temperatur ORDER BY createdat;"
# df = pd.read_sql(query, conn)

# px.line(df,x=df['createdat'],y=df['value'])

In [8]:

def resample_data_one_hour(data):


    # Convert to DataFrame
    df = pd.DataFrame(data)
    df['createdAt'] = pd.to_datetime(df['createdAt'])
    df['value'] = df['value'].astype(float)

    # Set datetime index
    df.set_index('createdAt', inplace=True)

    # Resample to 1-hour intervals using mean
    resampled_df = df.resample('1h').mean().dropna()

    # Convert back to list[dict]
    resampled_data = [
    {'createdAt': ts.replace(tzinfo=None).isoformat() + 'Z', 'value': f"{val:.2f}"}
    for ts, val in resampled_df.itertuples()
    ]
    return resampled_data

In [ ]:

sensor_dict = get_sensor_names_ids()
#Get Time
now = datetime.now(timezone.utc) 
two_weeeks_ago = now - timedelta(weeks=2) 
iso_now = now.isoformat().replace('+00:00','Z')
iso_two_weeks_ago = two_weeeks_ago.isoformat().replace('+00:00','Z')

#-- Data initalization --
for name, id in sensor_dict.items():
    #Create Tables
    create_table(name)
    #Get Data for Tabel
    data = get_data(sensor_id=id,
                fromDate=iso_two_weeks_ago,
                toDate=iso_now)
    #Insert Data into table
    data = resample_data_one_hour(data)
    bulk_insert_into_table(table_name=name,
                            data=data)


create_table(table_name='Temperatur')

id = sensor_dict['Temperatur']
data = []
for iteration in range(10):
    print(f"From: {iso_two_weeks_ago} To: {iso_now}")
    
    new_data = get_data(sensor_id=id, fromDate=iso_two_weeks_ago, toDate=iso_now)
    data.extend(new_data)

    # Extract and parse latest time as datetime object
    last_time_str = new_data[-1]['createdAt'].replace('+00:00', 'Z')
    last_time = datetime.strptime(last_time_str, "%Y-%m-%dT%H:%M:%S.%fZ").replace(tzinfo=timezone.utc)
    
    print(f'{last_time.isoformat()}')

    # Update range for next loop
    iso_now = last_time.isoformat().replace('+00:00', 'Z')
    iso_two_weeks_ago = (last_time - timedelta(weeks=2, seconds=10)).isoformat().replace('+00:00', 'Z')
    print(f"*UPDATED* From: {iso_two_weeks_ago} To: {iso_now}")

data = resample_data_one_hour(data)
bulk_insert_into_table('Temperatur',data=data)

In [9]:
query = "SELECT * FROM public.temperatur ORDER BY createdat;"
df = pd.read_sql(query, conn)

C:\Users\bnjmn\AppData\Local\Temp\ipykernel_14620\834547581.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [10]:
df

,createdat,value
0,2025-04-18 16:00:00+00:00,29.90
1,2025-04-18 17:00:00+00:00,30.35
2,2025-04-18 18:00:00+00:00,29.45
3,2025-04-18 19:00:00+00:00,28.61
4,2025-04-18 20:00:00+00:00,28.46
...,...,...
1687,2025-06-29 06:00:00+00:00,25.45
1688,2025-06-29 07:00:00+00:00,25.26
1689,2025-06-29 08:00:00+00:00,25.13
1690,2025-06-29 09:00:00+00:00,25.04


In [35]:
px.line(df, x=df['createdat'], y=df['value'])

In [36]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.statespace.sarimax import SARIMAX

query = "SELECT * FROM public.temperatur ORDER BY createdat;"
df = pd.read_sql(query, conn)

df['createdat'] = pd.to_datetime(df['createdat'])
df.set_index('createdat', inplace=True)
df = df.asfreq('h')  # enforce hourly frequency
df['value'] = pd.to_numeric(df['value'], errors='coerce')

train = df.iloc[:-48]
test = df.iloc[-48:]

model = SARIMAX(train['value'],
                order=(2, 1, 2),
                seasonal_order=(1, 1, 1, 24),
                enforce_stationarity=False,
                enforce_invertibility=False)

model_fit = model.fit(disp=False)

forecast = model_fit.forecast(steps=48)
forecast.index = test.index  # Align forecast index to test index

# Prepare plot DataFrame
train_df = train.reset_index()
test_df = test.reset_index()
train_df['type'] = 'Train'
test_df['type'] = 'Test'
plot_df = pd.concat([train_df, test_df], ignore_index=True)
plot_df.rename(columns={'createdat': 'timestamp'}, inplace=True)

forecast_df = forecast.to_frame(name='value').reset_index()
forecast_df.rename(columns={'createdat': 'timestamp'}, inplace=True)
forecast_df['type'] = 'Forecast'

combined = pd.concat([plot_df, forecast_df], ignore_index=True)

fig = px.line(combined, x='timestamp', y='value', color='type',
              title='Hourly Temperature Forecast with SARIMA',
              labels={'value': 'Temperature (°C)', 'timestamp': 'Time', 'type': 'Data'})

fig.update_layout(template='plotly_white', hovermode='x unified')
fig.show()


C:\Users\bnjmn\AppData\Local\Temp\ipykernel_31580\2493065405.py:6: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [37]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.statespace.sarimax import SARIMAX

query = "SELECT * FROM public.temperatur ORDER BY createdat;"
df = pd.read_sql(query, conn)

df['createdat'] = pd.to_datetime(df['createdat'])
df.set_index('createdat', inplace=True)
df = df.asfreq('h')  # enforce hourly frequency
df['value'] = pd.to_numeric(df['value'], errors='coerce')

train = df.iloc[:-48]
test = df.iloc[-48:]

model = SARIMAX(train['value'],
                order=(2, 1, 2),
                seasonal_order=(1, 1, 1, 24),
                enforce_stationarity=False,
                enforce_invertibility=False)

model_fit = model.fit(disp=False)

forecast = model_fit.forecast(steps=48)
forecast.index = test.index  # Align forecast index to test index

# Prepare plot DataFrame
train_df = train.reset_index()
test_df = test.reset_index()
train_df['type'] = 'Train'
test_df['type'] = 'Test'
plot_df = pd.concat([train_df, test_df], ignore_index=True)
plot_df.rename(columns={'createdat': 'timestamp'}, inplace=True)

forecast_df = forecast.to_frame(name='value').reset_index()
forecast_df.rename(columns={'createdat': 'timestamp'}, inplace=True)
forecast_df['type'] = 'Forecast'

combined = pd.concat([plot_df, forecast_df], ignore_index=True)

fig = px.line(combined, x='timestamp', y='value', color='type',
              title='Hourly Temperature Forecast with SARIMA',
              labels={'value': 'Temperature (°C)', 'timestamp': 'Time', 'type': 'Data'})

fig.update_layout(template='plotly_white', hovermode='x unified')
fig.show()


C:\Users\bnjmn\AppData\Local\Temp\ipykernel_31580\2493065405.py:6: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [38]:
# Number of days to show from history
history_days = 5
forecast_hours = 48  # 2 days

# Filter last 5 days from historical data
start_date = df.index[-1] - pd.Timedelta(days=history_days)
historical_recent = df.loc[start_date:]

# Forecast future 2 days
model = SARIMAX(df['value'],
                order=(2, 1, 2),
                seasonal_order=(1, 1, 1, 24),
                enforce_stationarity=False,
                enforce_invertibility=False)
model_fit = model.fit(disp=False)
forecast = model_fit.forecast(steps=forecast_hours)

# Create future datetime index starting after last data point
last_date = df.index[-1]
future_index = pd.date_range(start=last_date + pd.Timedelta(hours=1),
                             periods=forecast_hours,
                             freq='h')
forecast.index = future_index

# Prepare dataframes for plot
hist_df = historical_recent.reset_index().rename(columns={'createdat': 'timestamp'})
hist_df['type'] = 'Historical'

forecast_df = forecast.to_frame(name='value').reset_index()
forecast_df.rename(columns={'index': 'timestamp'}, inplace=True)
forecast_df['type'] = 'Forecast'

combined = pd.concat([hist_df, forecast_df], ignore_index=True)

# Plot with plotly
fig = px.line(combined, x='timestamp', y='value', color='type',
              title='Last 5 Days Historical and 2 Days Forecast Temperature',
              labels={'value': 'Temperature (°C)', 'timestamp': 'Time', 'type': 'Data'})

fig.update_layout(template='plotly_white', hovermode='x unified')
fig.show()


In [ ]:

# import dash
# from dash import dcc, html, Input, Output
# import plotly.express as px
# import pandas as pd

# # --- Load and preprocess data ---
# query = "SELECT * FROM public.temperatur ORDER BY createdat;"
# df = pd.read_sql(query, conn)

# df['createdat'] = pd.to_datetime(df['createdat'])
# df.set_index('createdat', inplace=True)
# df = df.asfreq('h')  # enforce hourly frequency
# df['value'] = pd.to_numeric(df['value'], errors='coerce')

# # --- Dash App ---
# app = dash.Dash(__name__)

# app.layout = html.Div([
#     html.H1("Historical Temperature Data"),

#     html.Div([
#         html.Label("Select Time Range:"),
#         dcc.Dropdown(
#             id='interval-dropdown',
#             options=[
#                 {'label': 'Last 1 Day', 'value': 1},
#                 {'label': 'Last 2 Days', 'value': 2},
#                 {'label': 'Last 1 Week', 'value': 7}
#             ],
#             value=1,
#             clearable=False,
#             style={'width': '200px'}
#         )
#     ], style={'padding': '10px'}),

#     dcc.Graph(id='historical-graph')
# ])

# @app.callback(
#     Output('historical-graph', 'figure'),
#     Input('interval-dropdown', 'value')
# )
# def update_historical_graph(days):
#     max_date = df.index.max()
#     min_date = max_date - pd.Timedelta(days=days)
#     hist_df = df.loc[min_date:max_date].reset_index()

#     fig = px.line(hist_df, x='createdat', y='value',
#                   title=f'Historical Temperature Data (Last {days} Day{"s" if days > 1 else ""})',
#                   labels={'value': 'Temperature (°C)', 'createdat': 'Time'})
#     fig.update_layout(template='plotly_white', hovermode='x unified')
#     return fig

# if __name__ == '__main__':
#     app.run(debug=True, port=8051)


In [17]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

import psycopg2  # or use your own `conn` if already created

# --- Dash App ---
app = dash.Dash(__name__)
app.title = "Temperature Dashboard"

# --- Table Query Mapping ---
query_map = {
    "Temperatur": "SELECT * FROM public.temperatur ORDER BY createdat;",
    "Luftfeuchtigkeit": "SELECT * FROM public.rel_luftfeuchte ORDER BY createdat;",
    "Beleuchtungsstärke": "SELECT * FROM public.beleuchtungsstärke ORDER BY createdat;",
    "UV-Intensität": "SELECT * FROM public.uv_intensität ORDER BY createdat;",
    "PM1": "SELECT * FROM public.pm1 ORDER BY createdat;",
    "PM2.5": "SELECT * FROM public.pm2_5 ORDER BY createdat;",
    "PM4": "SELECT * FROM public.pm4 ORDER BY createdat;",
    "PM10": "SELECT * FROM public.pm10 ORDER BY createdat;",
}

# --- Load initial data for forecast (e.g., Temperatur) ---
initial_query = query_map["Temperatur"]
df = pd.read_sql(initial_query, conn)
df['createdat'] = pd.to_datetime(df['createdat'])
df.set_index('createdat', inplace=True)
df = df.asfreq('h')
df['value'] = pd.to_numeric(df['value'], errors='coerce')

# Forecast last 5 days + next 2 days
history_days = 5
forecast_hours = 48

start_date = df.index[-1] - pd.Timedelta(days=history_days)
historical_recent = df.loc[start_date:]

model = SARIMAX(df['value'],
                order=(2, 1, 2),
                seasonal_order=(1, 1, 1, 24),
                enforce_stationarity=False,
                enforce_invertibility=False)
model_fit = model.fit(disp=False)
forecast = model_fit.forecast(steps=forecast_hours)

future_index = pd.date_range(start=df.index[-1] + pd.Timedelta(hours=1),
                             periods=forecast_hours, freq='h')
forecast.index = future_index

# Combine for forecast graph
hist_df = historical_recent.reset_index().rename(columns={'createdat': 'timestamp'})
hist_df['type'] = 'Historical'

forecast_df = forecast.to_frame(name='value').reset_index()
forecast_df.rename(columns={'index': 'timestamp'}, inplace=True)
forecast_df['type'] = 'Forecast'

combined_forecast_df = pd.concat([hist_df, forecast_df], ignore_index=True)

# --- Layout ---
app.layout = html.Div([
    html.H1("Environmental Dashboard", style={'textAlign': 'center'}),

    html.Div([
        html.H3("Pure Historical Data"),

        html.Div([
            html.Label("Select Data Source:"),
            dcc.Dropdown(
                id='table-dropdown',
                options=[{'label': name, 'value': name} for name in query_map],
                value='Temperatur',
                style={'width': '300px'}
            ),
        ], style={'marginBottom': '10px'}),

        html.Div([
            html.Label("Select Interval:"),
            dcc.Dropdown(
                id='interval-dropdown',
                options=[
                    {'label': 'Last 1 Day', 'value': 1},
                    {'label': 'Last 2 Days', 'value': 2},
                    {'label': 'Last 1 Week', 'value': 7}
                ],
                value=1,
                clearable=False,
                style={'width': '200px'}
            ),
        ], style={'marginBottom': '20px'}),

        dcc.Graph(id='historical-graph')
    ], style={'marginBottom': '50px'}),

    html.Div([
        html.H3("Forecast (Last 5 Days + Next 2 Days)"),
        dcc.Graph(
            id='forecast-graph',
            figure=px.line(
                combined_forecast_df,
                x='timestamp', y='value', color='type',
                title='Temperature Forecast with SARIMA',
                labels={'value': 'Temperature (°C)', 'timestamp': 'Time', 'type': 'Data'}
            ).update_layout(template='plotly_white', hovermode='x unified')
        )
    ])
])

# --- Callback ---
@app.callback(
    Output('historical-graph', 'figure'),
    [Input('table-dropdown', 'value'),
     Input('interval-dropdown', 'value')]
)
def update_historical_graph(selected_table, days):
    query = query_map[selected_table]
    df = pd.read_sql(query, conn)
    df['createdat'] = pd.to_datetime(df['createdat'])
    df.set_index('createdat', inplace=True)
    df = df.asfreq('h')
    df['value'] = pd.to_numeric(df['value'], errors='coerce')

    max_date = df.index.max()
    min_date = max_date - pd.Timedelta(days=days)
    hist_df = df.loc[min_date:max_date].reset_index()

    fig = px.line(hist_df, x='createdat', y='value',
                  title=f'{selected_table} (Last {days} Day{"s" if days > 1 else ""})',
                  labels={'value': selected_table, 'createdat': 'Time'})
    fig.update_layout(template='plotly_white', hovermode='x unified')
    return fig

# --- Run ---
if __name__ == '__main__':
    app.run(debug=True, port=8051)


C:\Users\bnjmn\AppData\Local\Temp\ipykernel_14620\3234481054.py:27: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



DatabaseError: Execution failed on sql: SELECT * FROM public.temperatur ORDER BY createdat;
could not receive data from server: Software caused connection abort (0x00002745/10053)

unable to rollback

In [ ]:
queries = ["SELECT * FROM public.temperatur ORDER BY createdat;",
           "SELECT * FROM public.rel_luftfeuchte ORDER BY createdat;",
           "SELECT * FROM public.beleuchtungsstärke ORDER BY createdat;",
           "SELECT * FROM public.uv_intensität ORDER BY createdat;",
           "SELECT * FROM public.pm1 ORDER BY createdat;",
           "SELECT * FROM public.pm2_5 ORDER BY createdat;",
           "SELECT * FROM public.pm4 ORDER BY createdat;",
           "SELECT * FROM public.pm10 ORDER BY createdat;"
           ]

df = pd.read_sql(query, conn)
px.line(df,x=df.createdat,y=df.value)

C:\Users\bnjmn\AppData\Local\Temp\ipykernel_31580\1149897907.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

